In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_predict, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing

In [2]:
# options
pd.options.display.max_columns = None # show all columns

In [3]:
def load_data():
    # paths to data
    data_dir = os.getcwd() + "/data/"
    X_train_path = data_dir + "train_augmented_2.csv"
    #X_test_path = data_dir + "X_test.csv"
    y_train_path = data_dir + "y_train.csv"
    regis_data_path = data_dir + "vehicle_registration_data_2018.csv"
    
    # load dataframes
    X_train = pd.read_csv(X_train_path, index_col='idx')
    X_train.drop(X_train.columns[0], axis=1, inplace=True) # drop repeated index column
    y_train = pd.read_csv(y_train_path, usecols=['idx','UNITS'], index_col='idx')
    #regis_data = pd.read_csv(regis_data_path)
    
    return X_train, y_train

In [15]:
def load_test():
    # paths to data
    data_dir = os.getcwd() + "/data/"
    X_test_path = data_dir + "X_test.csv"
    
    # load dataframes
    X_test = pd.read_csv(X_test_path, index_col='idx')
    X_test.drop(X_test.columns[0], axis=1, inplace=True) # drop repeated index column
    
    return X_test

## Grouping by unique products

In [4]:
def group_by_product():
    # groupby product skew maybe useful???
    temp = X_train.groupby(['CATEGORY','TIER','SPEED_RATING_CODE','RIM_DIAMETER_SIZE_CODE']).groups
    for k,values in temp.items():
        print ("Product Skew: ",k)
        for v in values:
            example = X_train.iloc[v]
            break
        break
    example

### CLEAN Null values

In [5]:
# Replace NaNs with 0
def clean_data(X):
    X.fillna(0, inplace=True)
    X.replace([-np.inf, np.inf, np.NaN],0, inplace=True)
    ## transform nominals to numeric codes
    for col in X:
        if X[col].dtype == 'object':
            X[col] = X[col].astype('category').cat.codes
    X[X.select_dtypes(np.int64).columns.values].astype(np.float64, inplace=True)

### K-means clustering 

In [6]:
def cluster_it():
    # Drop rows containing outliers in their numerical columns if desired
    #print ("Dropping outliers...")
    #df = df[(np.abs(stats.zscore(df[cols_to_convert])) < 3).all(axis=1)]

    # need the number of labels for categorization
    num_labels = 5

    # Fit a kmeans model to the column
    #mat = df[col].values.reshape(-1,1) # convert dataframe col to matrix
    mat = X_train.loc[:,'Monthly_Top_1_Customer_Total_Sales':].values

    km = KMeans(n_clusters=num_labels)
    km.fit(mat)
    labels = km.labels_ # Get cluster assignment labels

    sales_cols = ['Monthly_Top_{}_Customer_Total_Sales'.format(i) for i in range(1,11)]
    
    # Format results as a DataFrame
    sales_n_labels = X_train.loc[:,'Monthly_Top_1_Customer_Total_Sales':]
    sales_n_labels['Clusters'] = labels
    #sales_n_labels.head()

###

In [7]:
def run_xgbr(X,y):
    print ("Running Grid Search CV...")

    
    gsc = GridSearchCV(
        estimator=XGBRegressor(),
        param_grid={
            'n_estimators': [50,100,150,200],
        },
        cv=5, scoring='neg_log_loss', verbose=2, n_jobs=7)

    print ("Fitting train data to xgbr...")

    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_

    xgbr = XGBRegressor(loss='quantile', 
                learning_rate=0.001,
                max_depth=5,
                n_estimators=best_params["n_estimators"],
                min_samples_split=3,
                random_state=7,
                max_features='log2',
                nthread=7,
                silent=0)

    print ("Cross validating the model...")

    scores = cross_val_score(xgbr, X, y, cv=5, scoring='neg_mean_squared_error')
    
    return xgbr, scores, best_params
    

In [25]:
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
# load data
# Get data
X, y = load_data()
clean_data(X)

# split data into train and test sets
seed = 7
test_size = 0.33

# 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed, shuffle=True)
#X_train_cv, y_train_cv = X_train.iloc[:25000], y_train.iloc[:25000]
X_train, y_train = X_train[:200000], y_train[:200000]

#model, scores, best_params = run_xgbr(X_train_sub,y_train_sub)
#print (best_params)

# fit model no training data
model = XGBRegressor(eval_metric="error", 
                learning_rate=0.01,
                max_depth=10,
                n_estimators=500,
                random_state=7,
                nthread=7,# was 12
                silent=0)
model.fit(X_train, y_train)

# make predictions for test data
rmse = sqrt(mean_squared_error(y_test, model.predict(X_test)))
rmse
#predictions = [round(value) for value in y_pred]

/home/joseph/.virtualenvs/rtw19-hacks/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


7.0065783621312985

In [26]:
# save it
import pickle
pickle.dump(model, open("best_xgbr.pickle.dat", "wb"))

In [ ]:
dot_data = tree.export_graphviz()

In [22]:
X_train.head()

,DC_ZIPCODE,Invoice_Year,Invoice_Week,CATEGORY,TIER,SPEED_RATING_CODE,RIM_DIAMETER_SIZE_CODE,WIDTH,HEIGHT,AVG_UNIT_WEIGHT,Invoice_Month,SELLING_PRICE,Monthly_Top_1_Customer_Zip,Monthly_Top_2_Customer_Zip,Monthly_Top_3_Customer_Zip,Monthly_Top_4_Customer_Zip,Monthly_Top_5_Customer_Zip,Monthly_Top_6_Customer_Zip,Monthly_Top_7_Customer_Zip,Monthly_Top_8_Customer_Zip,Monthly_Top_9_Customer_Zip,Monthly_Top_10_Customer_Zip,Monthly_Top_1_Customer_Total_Sales,Monthly_Top_2_Customer_Total_Sales,Monthly_Top_3_Customer_Total_Sales,Monthly_Top_4_Customer_Total_Sales,Monthly_Top_5_Customer_Total_Sales,Monthly_Top_6_Customer_Total_Sales,Monthly_Top_7_Customer_Total_Sales,Monthly_Top_8_Customer_Total_Sales,Monthly_Top_9_Customer_Total_Sales,Monthly_Top_10_Customer_Total_Sales,DC_ZIPCODE_POPULATION,population_employed,population_commuting,total_household_income,median_household_income,mean_household_income
idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
667801,33025,2017,48,5,1,11,18.0,10.00,27.100000,25.60,11.0,121.00,33166.0,33311.0,33166.0,33030.0,33157.0,33020.0,33309.0,33023.0,33186.0,33165.0,659232.0,247476.0,309714.0,197820.0,160644.0,118488.0,166718.0,90634.0,113086.0,125206.0,68450.0,55115.0,35882.0,23332.0,55708.0,63231.0
1348304,75006,2016,10,2,3,9,17.0,10.71,31.153333,39.57,3.0,86.00,76051.0,75287.0,74701.0,75147.0,75109.0,75090.0,75223.0,75103.0,75160.0,75071.0,787606.0,410248.0,144084.0,155886.0,160904.0,122412.0,84094.0,135888.0,86758.0,75556.0,50788.0,39258.0,26621.0,17780.0,57415.0,76769.0
778668,36117,2016,25,2,2,2,16.0,9.25,31.730000,52.45,6.0,114.50,36117.0,36093.0,36092.0,36801.0,36109.0,35010.0,36401.0,36066.0,36106.0,36054.0,307800.0,171382.0,137858.0,209402.0,140372.0,130720.0,167868.0,121300.0,88310.0,94864.0,53172.0,41860.0,26916.0,21945.0,60884.0,80423.0
1720363,91730,2016,16,2,3,9,16.0,10.71,30.630000,37.88,4.0,83.75,91710.0,91730.0,92335.0,92504.0,92553.0,92284.0,92508.0,91740.0,91710.0,92225.0,469516.0,167044.0,191838.0,109626.0,91366.0,104612.0,108674.0,97932.0,173932.0,100548.0,70545.0,55425.0,31793.0,23306.0,68716.0,79265.0
894841,39218,2017,15,2,1,8,17.0,9.25,30.870000,38.00,4.0,138.00,39208.0,39218.0,39648.0,39272.0,39201.0,71291.0,39090.0,71270.0,39601.0,39350.0,158678.0,146676.0,113862.0,116466.0,82600.0,149986.0,107578.0,106010.0,67976.0,97868.0,7107.0,5304.0,3542.0,2461.0,46242.0,55539.0


In [21]:
X_test.head()

,DC_ZIPCODE,Invoice_Year,Invoice_Week,CATEGORY,TIER,SPEED_RATING_CODE,RIM_DIAMETER_SIZE_CODE,WIDTH,HEIGHT,AVG_UNIT_WEIGHT,Invoice_Month,SELLING_PRICE,Monthly_Top_1_Customer_Zip,Monthly_Top_2_Customer_Zip,Monthly_Top_3_Customer_Zip,Monthly_Top_4_Customer_Zip,Monthly_Top_5_Customer_Zip,Monthly_Top_6_Customer_Zip,Monthly_Top_7_Customer_Zip,Monthly_Top_8_Customer_Zip,Monthly_Top_9_Customer_Zip,Monthly_Top_10_Customer_Zip,Monthly_Top_1_Customer_Total_Sales,Monthly_Top_2_Customer_Total_Sales,Monthly_Top_3_Customer_Total_Sales,Monthly_Top_4_Customer_Total_Sales,Monthly_Top_5_Customer_Total_Sales,Monthly_Top_6_Customer_Total_Sales,Monthly_Top_7_Customer_Total_Sales,Monthly_Top_8_Customer_Total_Sales,Monthly_Top_9_Customer_Total_Sales,Monthly_Top_10_Customer_Total_Sales
idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,19355,2017,51,4,3,4,12.0,6.18,21.77,14.24,12.0,31.500000,19802.0,17111.0,19355.0,19508.0,19382.0,19335.0,19341.0,17566.0,19454.0,17104.0,452002.0,219816.0,324038.0,147536.0,168540.0,133754.0,129520.0,103130.0,122898.0,106798.0
1,19355,2017,15,4,1,9,13.0,6.97,22.68,12.80,4.0,41.000000,19802.0,17111.0,19508.0,19382.0,19335.0,17104.0,17566.0,19341.0,19810.0,19702.0,655098.0,93626.0,89266.0,118444.0,88896.0,97508.0,65778.0,71150.0,59238.0,66294.0
2,19355,2017,16,4,1,9,13.0,6.97,22.68,12.80,4.0,43.000000,19802.0,17111.0,19508.0,19382.0,19335.0,17104.0,17566.0,19341.0,19810.0,19702.0,655098.0,93626.0,89266.0,118444.0,88896.0,97508.0,65778.0,71150.0,59238.0,66294.0
3,19355,2017,43,4,1,9,13.0,6.97,22.68,12.80,10.0,45.000000,19802.0,17111.0,19355.0,19335.0,19382.0,19508.0,17104.0,19454.0,19341.0,17566.0,657996.0,183524.0,366208.0,119630.0,138148.0,120654.0,126132.0,120934.0,78346.0,83514.0
4,19355,2017,28,4,1,8,13.0,7.40,23.23,15.97,7.0,53.266667,19802.0,17111.0,19508.0,19335.0,19382.0,17566.0,17104.0,19702.0,19355.0,19341.0,666558.0,149832.0,100894.0,92446.0,97040.0,97112.0,80830.0,85096.0,168750.0,59476.0


In [20]:
X_test = load_test()
clean_data(X_test)
model.predict(X_test).round().shape
#upload_leaderboard(result)

ValueError: feature_names mismatch: ['DC_ZIPCODE', 'Invoice_Year', 'Invoice_Week', 'CATEGORY', 'TIER', 'SPEED_RATING_CODE', 'RIM_DIAMETER_SIZE_CODE', 'WIDTH', 'HEIGHT', 'AVG_UNIT_WEIGHT', 'Invoice_Month', 'SELLING_PRICE', 'Monthly_Top_1_Customer_Zip', 'Monthly_Top_2_Customer_Zip', 'Monthly_Top_3_Customer_Zip', 'Monthly_Top_4_Customer_Zip', 'Monthly_Top_5_Customer_Zip', 'Monthly_Top_6_Customer_Zip', 'Monthly_Top_7_Customer_Zip', 'Monthly_Top_8_Customer_Zip', 'Monthly_Top_9_Customer_Zip', 'Monthly_Top_10_Customer_Zip', 'Monthly_Top_1_Customer_Total_Sales', 'Monthly_Top_2_Customer_Total_Sales', 'Monthly_Top_3_Customer_Total_Sales', 'Monthly_Top_4_Customer_Total_Sales', 'Monthly_Top_5_Customer_Total_Sales', 'Monthly_Top_6_Customer_Total_Sales', 'Monthly_Top_7_Customer_Total_Sales', 'Monthly_Top_8_Customer_Total_Sales', 'Monthly_Top_9_Customer_Total_Sales', 'Monthly_Top_10_Customer_Total_Sales', 'DC_ZIPCODE_POPULATION', 'population_employed', 'population_commuting', 'total_household_income'

In [ ]:
def submit_score(predictions, team_key):
    """
    Submit your predictions for scoring

    Args:
        predictions (DataFrame): Pandas DataFrame containing the following required
            column:
                1. idx (int) - The unique identifier for each observation
                2. predictions (float) - Your predicted value
        team_key (str): Your team's unique identifier

    Returns:
        Response: Flask Response object. See the Response.text field to get the score
            from your latest submission. Your best score will be reflected on the
            leaderboard
    """

    import requests
    import json
    import numpy
    def default(o):
        if isinstance(o, numpy.int64):
            return int(o)
        raise TypeError

    API_ENDPOINT = "http://coe-hackathon-dot-atd-fn-anacoe-dev.appspot.com/submitscore"
    payload = {
        "team_key": team_key,
        "data": predictions.loc[:, ["idx", "predictions"]].to_dict(orient="records")
    }
    resp = requests.post(
        API_ENDPOINT,
        data=json.dumps(payload, default=default),
        headers={'Content-Type': 'application/json'}
    )
    
    if resp.status_code == 404:
        print(resp.json()['error'])
        return None
    
    elif resp.status_code != 200:
        raise ValueError('There was an error processing your request: '
                         '\n{}'.format(resp.text))
        return None
    else:
        score = resp.json()['score']
        print('Submission successful! Your score was \n{}'.format(score))
        return score

def upload_leaderboard(upload_predictions):
    teamkey = '$pbkdf2-sha512$25000$BOA8p/S.N6aUktK6d6415g$5KWTQmlXfm30T4H/c/Vo8Tiacfqd/qst5n3nU9JLSyV3fqZxerFbbCYYwCc3KVlOyv1pnvGJga7CU/CMzM6yNw'
    upload_dataframe = pd.DataFrame()
    upload_dataframe['idx'] = range(upload_predictions.size)
    upload_dataframe['predictions'] = np.array(upload_predictions)
    upload_dataframe['predictions'] = upload_dataframe['predictions']
    upload_dataframe.shape
    submit_score(upload_dataframe[['idx','predictions']],teamkey)